<a href="https://colab.research.google.com/github/manish254/p_pl_dl/blob/main/Handbrake_by_using_Rclone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <<----<strong>Install HandBrake</strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
HANDBRAKE = True #@param {type:"boolean"}
#@markdown Check AUTO_RECONNECT to prevent notebook from disconnecting!

from os import makedirs
makedirs("/content/temp/HandbrakeTemp", exist_ok=True)

if HANDBRAKE == True:
    !wget -qq https://github.com/vot/ffbinaries-prebuilt/releases/download/v4.2.1/ffmpeg-4.2.1-linux-64.zip
    !rm -f ffmpeg-4.2.1-linux-64.zip
    !add-apt-repository ppa:stebbins/handbrake-releases -y
    !apt-get install -y handbrake-cli

if AUTO_RECONNECT:
    import IPython
    from google.colab import output

    display(IPython.display.Javascript('''
    function ClickConnect(){
      btn = document.querySelector("colab-connect-button")
      if (btn != null){
        console.log("Click colab-connect-button");
        btn.click()
        }

      btn = document.getElementById('ok')
      if (btn != null){
        console.log("Click reconnect");
        btn.click()
        }
      }

    setInterval(ClickConnect, 60000);
    '''))


# <center >Rclone Mount </center>
<center><img src="https://forum.rclone.org/uploads/default/original/2X/d/da6ccb2784ff3fa73d9339300530e0aae4d6cebd.png" alt="rclone forum" width=150></center>




In [ ]:
#@markdown <h3>📝 Note: Run this before using Rclone.</h3>
#@markdown <h4>📝 Upload an rclone profile</4>
Setup_Time_Zone = False

import os; from google.colab import files; from IPython.display import HTML, clear_output

def upload_conf():
  try:
    display(HTML("<h2 style=\"font-family:Trebuchet MS;color:#446785;\">Please upload the config file of rclone (rclone.conf) from your computer.</h2><br>"))
    UploadConfig = files.upload().keys()
    clear_output(wait=True)
    if len(UploadConfig) == 0:
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been cancelled during upload file.</h2><br></center>"))
    elif len(UploadConfig) == 1:
      for fn in UploadConfig:
        if os.path.isfile("/content/" + fn) == True:
          os.environ["rclone_conf"] = fn
          !mv -f "$rclone_conf" /root/.rclone.conf
          !chmod 666 /root/.rclone.conf
          if Setup_Time_Zone == True:
            !sudo dpkg-reconfigure tzdata
            clear_output(wait=True)
          if os.path.isfile("/usr/bin/rclone") == True:
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
          else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
        else:
          return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">File upload has been failed during upload file.</h2><br></center>"))
    else:
      for fn in UploadConfig:
        os.environ["rclone_conf"] = fn
        !rm -f "$rclone_conf"
      return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Please uploading only one file at a time.</h2><br></center>"))
  except:
    clear_output(wait=True)
    return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during upload file.</h2><br></center>"))

upload_conf()

In [ ]:
# ============================= FORM ============================= #
#@markdown <h3>📝 Run this to mount the drive with rclone</h3>
rclone_config_name = "Bdrive" #@param {type:"string"}
local_mount_location = "/content/Bdrive" #@param {type:"string"}
#@markdown The default qBittorrent download location is '/content/udrive/qBittorrent/' (they get downloaded to '/content/qBittorrent/' first)
# ============================= FORM ============================= #

!sudo apt-get install fuse3

# Rest of the code
import time
import os

# clear nohup
open("nohup.out", 'w').close()


# unmount first

!fusermount -u $local_mount_location 2>/dev/null


# mount without waiting for the command to complete
!mkdir $local_mount_location 2>/dev/null
!nohup rclone mount $rclone_config_name: $local_mount_location --buffer-size 96M &



# Show the output that was written to nohup
time.sleep(3)
f = open(r"nohup.out", "r")
nohupText = f.read()
f.close()


dirs = os.listdir(local_mount_location)

if len(dirs) > 0:
  clear_output(wait=True)
  print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)

else:
  print("\n\nNot succeeded. No files or directories in mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
  print("log:\n", nohupText)

# <center>***HandBrake*** </center>
# <Center> <img src="https://handbrake.fr/img/logo.png" width="40"></center>

In [ ]:
#@title <center> HandBrake Configuration </center>
#@markdown > Select Mode ( Batch conversion/ Single File)
MODE = "SINGLE" #@param ["SINGLE", "BATCH"]

#@markdown ---
SOURCE = "/content/mydrive/test.mp4" #@param {type:"string"}
DESTINATION = "/content/mydrive/" #@param {type:"string"}
FORMAT = "mp4" #@param ["mp4", "mkv"]
RESOLUTION = "1080p" #@param ["480p", "720p", "1080p"]
Encoder = "x265" #@param ["x264", "x265"]
Encoder_Preset = "ultrafast" #@param ["ultrafast", "faster", "fast", "medium", "slow", "slower"]
#@markdown Choose Constant Quality Rate [Lower = Higher Quality/
#@markdown Larger File Size]
CQ = 17 #@param {type:"slider", min:10, max:30, step:1}
Additional_Flags = "" #@param {type:"string"}
#@markdown ---
########################################################

formats = ('.mkv','.mp4','.ts','.avi','.mov','.wmv')

######## Renames the file ########
def fileName(fPath):
        tName = fPath.split('/')[-1]
        if tName.endswith('ts'):
          tName = '[HANDY] ' + tName[:-3] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'
        else:
          tName = '[HANDY] ' + tName[:-4] + f' [{RESOLUTION}] [{Encoder}].{FORMAT}'
        return tName

def set_resolution():
  global w,h,flags
  if RESOLUTION == "480p":
    w, h = "854" , "480"
  elif RESOLUTION == "720p":
    w, h = "1280" , "720"
  elif RESOLUTION=="1080p":
    w, h = "1920" , "1080"

def addFlags():
  global flags
  flags = f" --encoder {Encoder}  --all-audio -s '0,1,2,3' --cfr --optimize --quality={CQ} --width={w} --height={h} --format={FORMAT} --encoder-preset={Encoder_Preset} "
  if Additional_Flags != "":
    flags += str(Additional_Flags)

set_resolution()
addFlags()

##### HandBrake and Direct Move #####
def runner(path):
    f_name = fileName(path)
    hTemp = f"/content/temp/HandbrakeTemp/{f_name}"
    !HandBrakeCLI -i "$path" -o "$hTemp" $flags

    if os.path.isfile(hTemp):
        print(f"\n\n********** Successfully converted {f_name}\n Now saving to Destination.....")
        dest = DESTINATION + '/' + f_name
        !mv "$hTemp" "$dest"
        if os.path.isfile(dest):
            print(f"\n\n********** Successfully saved {f_name} to Destination")

########## Check Mode ########
if MODE == "BATCH":
    os.makedirs(DESTINATION, exist_ok=True)
    if SOURCE.endswith('/'):
        pass
    else:
        SOURCE += '/'
    filesList = os.listdir(SOURCE + '.')
    if os.path.isfile(SOURCE + 'processed_db.txt'):
        pass
    else:
        with open((SOURCE + 'processed_db.txt'), 'w') as fb:
            fb.write("Do not delete this file until all files have been processed!\n")
            fb.close()
    with open((SOURCE + 'processed_db.txt'), "r+") as filehandle:
        processedList = [x.rstrip() for x in filehandle.readlines()]

        print('<<<<<<<<<<<<<<<<<< Starting Conversion in Batch mode. >>>>>>>>>>>>>>>>>>')

        for currentFile in filesList:
            if currentFile.endswith(formats):
                if currentFile not in processedList:
                    currentPath = SOURCE + currentFile
                    print(f'\n\n**************** Current File to process: {currentFile}')
                    runner(currentPath)
                    filehandle.write(currentFile+'\n')
        filehandle.close()

else:
    if SOURCE.endswith(formats):
        runner(SOURCE)
    else:
        print("Are you sure you have selected the correct file??")
